# 4. Softmax Regression

### When do we use Softmax Regression?

Multinomial Classification 문제를 해결할 때(n개의 값을 예측할 때), Logistic을 여러번 돌려서 구현할 수도 있지만 번거롭다.

Softmax Regression은 Logistic Classification에서의 Weight Function을 합쳐서, 한 번에 계산할 수 있도록 만들어준다.

예를 들어, 여러 데이터들이 있을 때 A/B/C로 구분하는 Task가 있다고 하자. 이 때, A or B / A or C / B or C 의 3가지 Logistic을 돌리는 것이 아니라, **A,B,C 각각에 대한 input을 받아서 어떤 것이 가장 Probable한지 확률**로서 접근하는 것이 Softmax Regression이다.

A가 될 확률이 0.7 / B가 될 확률이 0.3 / C가 될 확률이 0.1 이런 식으로 나오면 되는 것이다.

### Cost Function

Cost Function은 기본적으로 예측한 답이 맞으면 낮은 값을 주고, 잘못 예측했을 때 Penalty의 개념으로 1을 주는 것이다.

**Cross Entropy**

Softmax Regression에서는 Logistic Regression의 Cost Function은 실질적으로 같은 것이다.

![soft](img/softcost.png)

[A값, B값, C값] = [1,0,0] 이라고 할 때, 1을 전달하면 y값이 무한대로 발산하고 0을 전달하면 0으로 수렴한다. 즉, 잘못 예측했을 때는 1을 전달하여 y를 무한대로 보내 Penalty를 주고 제대로 예측했을 때는 0을 전달해  y를 0으로 보내 맞다는 것을 알려주는 것이다.

In [2]:
import tensorflow as tf

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [3]:
#외부의 값이 들어올 건데(placeholder), float자료형이고 4차원 데이터야!
X = tf.placeholder("float", [None, 4])

#외부의 값이 들어올 건데(placeholder), float자료형이고 3차원 데이터야!
Y = tf.placeholder("float", [None, 3])

#3개로 분류한다는 뜻이야
nb_classes = 3

#4(입력데이터) X nb_classes(3개)만큼의 차원에서 Weight을 줄거야
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')

#bias도 nb_classes랑 비슷하게 줄거고
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

In [11]:
#가설설정
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

#크로스 엔트로피
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    #arg_max : 몇 번째의 Argument가 가장 큰가요?에 대한 답을 줌
    a = sess.run(hypothesis, feed_dict={X: [[1,11,7,9]]})
    print(a, sess.run(tf.arg_max(a, 1)))

(0, 3.3390861)
(200, 0.53559661)
(400, 0.44353098)
(600, 0.37142956)
(800, 0.30192119)
(1000, 0.2364492)
(1200, 0.21347634)
(1400, 0.19517291)
(1600, 0.17965384)
(1800, 0.16633877)
(2000, 0.15479784)
(array([[  1.19744390e-02,   9.88015234e-01,   1.03607545e-05]], dtype=float32), array([1]))


### Fancy version

In [ ]:
logits = tf.matmul(X, W) + b